In [9]:
# Lab 7 Learning rate and Evaluation
# Test Data : MNIST
import tensorflow as tf
import matplotlib.pyplot as plt
import random

tf.set_random_seed(777) # for reproductibility


In [20]:
# import MNIST Data

# Check out https://www.tensorflow.org/get_started/mnist/beginners for
# more information about the mnist dataset

from tensorflow.examples.tutorials.mnist import input_data

# Data import One-Hot Encoding
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [27]:
mnist_classes = 784
nb_classes = 10
l1_classes = 60
l2_classes = 60

# MNIST data image of shape 28 * 28 = 784
X = tf.placeholder(tf.float32, [None, mnist_classes])
# 0 - 9 digits recognition = 10 classes
Y = tf.placeholder(tf.float32, [None, nb_classes])

# Layer 1
# 모델 수립을 위한 Weight, bias 선언
# Deep -> Many Layer
# Wide -> Many Output
# 첫 Layer 부터 무조건 출력값을 유지할 필요는 없음
# 실제 Lab9-3의 예제에서도, 실제 값에 구애받지 않고 출력값을 늘려서
# W, B를 산출 한 후, 이용 -> 최종 산출값만 맞춰서 적용
#W1 = tf.Variable(tf.random_normal([784, nb_classes]))
#For Wide
#W1 = tf.Variable(tf.random_normal([784, 784]))
#For Deep
W1 = tf.Variable(tf.random_normal([mnist_classes, l1_classes]))
#b1 = tf.Variable(tf.random_normal([nb_classes]))
#b1 = tf.Variable(tf.random_normal([784]))
b1 = tf.Variable(tf.random_normal([l1_classes]))
#Layer1 Result
#matmul [None, 784] * [784, 10] = [None, 10]
#layer1Res = tf.nn.softmax(tf.matmul(X, W1) + b1)
layer1Res = tf.nn.relu(tf.matmul(X, W1) + b1)
#layer1Res = tf.matmul(X, W1) + b1


In [31]:
#Layer2
#Layer를 하나 더 한다는 의미는, 실제 hypothesis에 도달하는 값에 양쪽layer
#의 값이 영향을 미쳐야 함.
W2 = tf.Variable(tf.random_normal([l1_classes, l2_classes]))
b2 = tf.Variable(tf.random_normal([l2_classes]))

#layer2Res = tf.nn.softmax(tf.matmul(layer1Res, W2) + b2)
layer2Res = tf.nn.relu(tf.matmul(layer1Res, W2) + b2)
#layer2Res = tf.matmul(layer1Res, W2) + b2

#Layer3
#Test를 위해 동일한 방식으로 Layer를 하나 추가
#Layer를 3개로 늘려서 처리했을 때
#기존 : 0.89, 현재 : 0.429 -> 예측 실패 9를 7로 예측
#TODO 실제로 출력값을 784로 맞추지 말고, 일단 줄여서 출력하고,
#이후 적정한 숫자를 찾아가는 방식은 어떨까?? 확인 필요함
W3 = tf.Variable(tf.random_normal([l2_classes, nb_classes]))
b3 = tf.Variable(tf.random_normal([nb_classes]))

# Hypothesis (using softmax)
#hypothesis = tf.nn.softmax(tf.matmul(layer1Res, W2) + b2)

# Layer 하나에서 산출된 값을 그냥 ADD했을 때에, 기존 방식보다 accracy 저하
# 기존 : 0.89, 현재 : 0.78
#hypothesis = tf.nn.softmax(layer1Res + layer2Res)

# 다시 Layer1의 출력값을 늘리고, Layer2의 INPUT으로 입력하여 사용하는 경우
# 기존 : 0.89, 현재 : 0.68 -> 굳이?
#hypothesis = tf.nn.softmax(tf.matmul(layer2Res, W3) + b3)
#hypothesis = tf.nn.relu(tf.matmul(layer2Res, W3) + b3)
logits = tf.matmul(layer2Res, W3) + b3

#cost 산출 하기 전의 hypothesis 를 산출하는 부분이 출력층으로 판단할 수 있음

# 해당 상태에서 reduce_mean 을 사용하면 nan 발생
#cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
#train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)
#train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)
train = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

# 결국 사용자를 통해 정해지는 hyperparameter가 굉장히 중요한 차이점을 만들어낸다.
# Layer가 늘어나면 learning_rate는 되도록이면 작게 설정

# Test model
prediction = tf.nn.softmax(logits)
is_correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
#is_correct = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
# Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [ ]:
# parameters
# epoch -> 입력된 데이터를 1번 모두 수행한 경우 1epoch
#num_epochs = 15
num_epochs = 15
batch_size = 100
num_iterations = int(mnist.train.num_examples / batch_size)

with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())
    # Training cycle
    for epoch in range(num_epochs):
        avg_cost = 0

        for i in range(num_iterations):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, cost_val = sess.run([train, cost], feed_dict={X: batch_xs, Y: batch_ys})
            avg_cost += cost_val / num_iterations

        print("Epoch: {:04d}, Cost: {:.9f}".format(epoch + 1, avg_cost))

    print("Learning finished")

    # Test the model using test sets
    print(
        "Accuracy: ",
        accuracy.eval(
            session=sess, feed_dict={X: mnist.test.images, Y: mnist.test.labels}
        ),
    )

    # Get one and predict
    r = random.randint(0, mnist.test.num_examples - 1)
    print("Label: ", sess.run(tf.argmax(mnist.test.labels[r : r + 1], 1)))
    print(
        "Prediction: ",
        #sess.run(tf.argmax(hypothesis, 1), feed_dict={X: mnist.test.images[r : r + 1]}),
        sess.run(tf.argmax(logits, 1), feed_dict={X: mnist.test.images[r : r + 1]}),
    )

#    plt.imshow(
#        mnist.test.images[r : r + 1].reshape(28, 28),
#        cmap="Greys",
#        interpolation="nearest",
#    )
#    plt.show()